# Import & Load

In [ ]:
# For Local
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

import aaanalysis as aa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

os.chdir('/Users/lynnshi/Library/CloudStorage/GoogleDrive-shiruosang@gmail.com/Meine Ablage/Thesis')
current_dir = os.getcwd()
print(current_dir)

In [ ]:
# substrate dataset
df_sub = pd.read_excel('data/ADAM_substrates.xlsx',sheet_name='ADAM17_sub')
df_nonsub = pd.read_excel('data/ADAM_substrates.xlsx',sheet_name='ADAM17_nonsub')
# reference dataset
df_ref = pd.read_excel('data/reference_set/SwissProt_human_type_I_TMP.xlsx')

# remove redundant term from raw data
from data_extractor.helper_functions import check_duplicate_entries
check_duplicate_entries(df_ref, 'Entry') # df_sub, 'final_entry' ; df_nonsub, 'final_entry'
#df_sub = df_sub.drop_duplicates(subset=['entry'])

from data.utils.dataframe_process_pipeline import *
# Here the df_type_I_ref_filtered is the ref dataset that is not in df_type_I_sub or df_type_I_nonsub
df_type_I_sub, df_type_I_nonsub, df_type_I_ref_filtered = process_type_I_TMP_ADAM17(df_sub, df_nonsub, df_ref)

df_sub_cs = df_type_I_sub[df_type_I_sub['final_cleavage_site'].apply(len) > 0].reset_index(drop=True).loc[:,['final_entry','final_cleavage_site','extracellular','transmembrane','intracellular']]
df_sub_no_cs = df_type_I_sub[df_type_I_sub['final_cleavage_site'].apply(len) == 0].reset_index(drop=True).loc[:,['final_entry','final_cleavage_site','extracellular','transmembrane','intracellular']]
df_nonsub = df_type_I_nonsub.loc[:,['final_entry','extracellular','transmembrane','intracellular']]

In [ ]:
from cleavage_site_predictor.window_slicer import WindowSlicer

slicer = WindowSlicer(window_sizes=[24])

positive_windows = slicer.generate_windows(df_sub_cs, window_type="positive", 
                                           enable_distance_filter=True, max_sequence_distance=150, max_spatial_distance=60.0,
                                           cd_hit_clustering=False, threshold=0.4, 
                                           show_topology=True,
                                           uniprot_id_col='final_entry',
                                           cleavage_sites_col='final_cleavage_site',
                                           extracellular_regions_col='extracellular',
                                           transmembrane_regions_col='transmembrane',
                                           intracellular_regions_col='intracellular')
negative_windows = slicer.generate_windows(df_sub_cs, window_type="negative", 
                                           enable_distance_filter=True, max_sequence_distance=150, max_spatial_distance=60.0,
                                           cd_hit_clustering=True, threshold=0.4, 
                                           show_topology=True,
                                           min_distance_from_cleavage=0,
                                           uniprot_id_col='final_entry',
                                           cleavage_sites_col='final_cleavage_site',
                                           extracellular_regions_col='extracellular',
                                           transmembrane_regions_col='transmembrane',
                                           intracellular_regions_col='intracellular')
print(positive_windows.shape)
print(negative_windows.shape)

cs_windows = pd.concat([positive_windows, negative_windows], ignore_index=True)
cs_windows = cs_windows[~cs_windows['sequence'].str.contains('X')].reset_index(drop=True)

In [ ]:
import pandas as pd

def df_to_fasta(df: pd.DataFrame, out_path: str | None = None) -> str:
    lines = []
    for _, row in df.iterrows():
        label = int(row["known_cleavage_site"]) if pd.notna(row["known_cleavage_site"]) else ""
        header = f">{row['entry']}_{int(row['center_position'])} {label}".rstrip()
        seq = str(row["sequence"]).replace(" ", "").strip()
        lines.append(header)
        lines.append(seq)
    fasta_str = "\n".join(lines)
    if out_path:
        with open(out_path, "w") as f:
            f.write(fasta_str + "\n")
    return fasta_str


cs_windows_24 = cs_windows[cs_windows['sequence'].str.len() == 24].reset_index(drop=True)
df_to_fasta(cs_windows_24, "0_output.fasta")                  

# Accessibility

In [ ]:
from cleavage_site_predictor.structural_analyzer.structural_profiler import DSSPProfiler

dssp_profiler = DSSPProfiler()
dssp_profiler.profile_dssp(cs_windows)

dssp_windows = dssp_profiler.profile_dssp(windows_df=cs_windows,
                                          show_original_data=True,
                                          include_rsa_mean=True)

dssp_profiler.plot_rsa_distribution(results=dssp_windows,
                                    labels=dssp_windows['known_cleavage_site'].values,
                                    title="Relative Solvent Accessibility (RSA) Distribution of CW vs NonCWs \n\nDistance: 0 aa between CW and nonCW | window size: 24 aa",
                                    rsa_col='rsa_mean',
                                    figsize=(12, 8))

ProsperousPlus


# CPP Train

In [ ]:
from cleavage_site_predictor.ml_construction.cpp_trainer import CPPTrainer

cpp_trainer = CPPTrainer(random_state=42, verbose=True)
cpp_results = cpp_trainer.train_and_evaluate(cs_windows)

In [ ]:
cpp_trainer.create_heatmap(
      metric='f1_05_mean',
      percentage=False,
      figsize=(16, 10),
      title='Evaluation CPP Features (Mean of F1 Score from 5 cross-validations) \nDistance: 0 aa between CW and nonCW\nLogodds Average + Soft voting (0.5 threshold)'
  )
cpp_trainer.create_heatmap(
      metric='balanced_accuracy_05_mean',
      percentage=False,
      figsize=(16, 10),
      title='Evaluation CPP Features (Mean of Balanced Accuracy Score from 5 cross-validations) \nDistance: 0 aa between CW and nonCW\nLogodds Average + Soft voting (0.5 threshold)'
  )
cpp_results_df = cpp_trainer.get_results_dataframe()
cpp_results_df.to_csv("0_cpp_results.csv", index=False)

# Train

In [ ]:
from cleavage_site_predictor.ml_construction.trainer import EnsembleTrainer

trainer = EnsembleTrainer()
results = trainer.run(cs_windows, 
                      model_type='SVM', 
                      use_group_based_split=True)

final_params = trainer.extract_final_results(results)

In [ ]:
trainer.plot_test_results(results,
                          title = 'Test Performance: Validation-Optimized vs Baselines\nWindow size: 28 aa | Distance: 0 aa between CW and nonCW | Full features \n Logodds Average + Soft Vote (Decision threshold: 0.5)',
                          show_only_0_5_threshold=True)

# Ablation Study

In [ ]:
cs_windows_24 = cs_windows[cs_windows['sequence'].str.len() == 24].reset_index(drop=True)

from cleavage_site_predictor.ml_construction.feature_ablation import FeatureAblationStudy

ablation_study = FeatureAblationStudy(random_state=42, verbose=True)
progressive_results = ablation_study.run_ablation_study(
        data=cs_windows_24,
        model_type='SVM',
        use_group_based_split=True, 
        outer_folds=5,  
        study_type='progressive'
    )

In [ ]:
ablation_study.plot_progressive_ablation_results(progressive_results, 
title = 'Progressive Feature Ablation: Impact of Removing Each Feature Group\n Baseline (All Features) vs. Remove One Group at a Time \nWindow size: 24 aa | Distance: 0 aa between CW and nonCW\n Logodds Average + Soft Vote (Use 0.5 default threshold)')

ablation_study.extract_results_to_csv(progressive_results, "0_ablation_results.csv")

# Substrates prediction

In [ ]:
slicer = WindowSlicer(window_sizes=[24])
sub_windows = slicer.generate_query_windows(df_sub_no_cs,
                              uniprot_id_col = 'final_entry',
                              extracellular_regions_col = 'extracellular',
                              transmembrane_regions_col = 'transmembrane',
                              intracellular_regions_col = 'intracellular',
                              enable_distance_filter = False, #4480
                              max_sequence_distance = 150,
                              max_spatial_distance = 60.0,
                              structure_source = "alphafold",
                              padding_char = 'X',
                              show_topology = True,
                              step_size = 1)

sub_windows = sub_windows[
    sub_windows['extracellular'].apply(lambda x: len(x) > 0) |
    (
        sub_windows['transmembrane'].apply(lambda x: len(x) > 0) &
        sub_windows['intracellular'].apply(lambda x: len(x) > 0)
    )
].reset_index(drop=True)
len(sub_windows)

sub_windows = sub_windows[~sub_windows['sequence'].str.contains('X')].reset_index(drop=True)


nonsub_windows = slicer.generate_query_windows(df_nonsub,
                              uniprot_id_col = 'final_entry',
                              extracellular_regions_col = 'extracellular',
                              transmembrane_regions_col = 'transmembrane',
                              intracellular_regions_col = 'intracellular',
                              enable_distance_filter = False, # 780
                              max_sequence_distance = 150,
                              max_spatial_distance = 60.0,
                              structure_source = "alphafold",
                              padding_char = 'X',
                              show_topology = True,
                              step_size = 1)
nonsub_windows = nonsub_windows[~nonsub_windows['sequence'].str.contains('X')].reset_index(drop=True)

In [ ]:
from cleavage_site_predictor.ml_construction.substrate_predictor import SubstratePredictor
predictor = SubstratePredictor(random_state=42,
                               filter_site_by_rsa=True,
                               rsa_min=0.4,
                               rsa_max=1.0)

aggregation_strategies = ['top_k_pooling', 'poisson_binomial']
predictor.fit_with_trainer_results(training_data = cs_windows, 
                                    trainer_results = final_params, 
                                    model_type = 'SVM')

nosub_predictions = predictor.predict_substrates(
            query_data=nonsub_windows,
            aggregation_strategies=aggregation_strategies
        )
sub_predictions = predictor.predict_substrates(
            query_data=sub_windows,
            aggregation_strategies=aggregation_strategies
        )

In [ ]:
calibration_results = predictor.evaluate_with_calibration(
    substrate_predictions=sub_predictions,
    nonsubstrate_predictions=nosub_predictions,
    cv_folds=5,
    random_state=42,
    bal_acc_weight=1,
    f1_weight=0,
    performance_title="Substrate Prediction: Aggregation Methods Comparison\n\nWindow size: 24 aa | Distance: 0 aa between CW and nonCW \nNo weighted matrices (Full Features except weighted matrices) \n RSA filter: 0.4-1.0 | 5 CV for threshold calibration",
    confusion_title="Confusion Matrices for Each Method | Calibrated Threshold\n\nWindow size: 24 aa \nNo weighted matrices (Full Features except weighted matrices) \nDistance: 0 aa between CW and nonCW | 5 CV for threshold calibration "
  )

In [ ]:
calibration_results